<a href="https://colab.research.google.com/github/satheeshnani/TensorFlow-Course/blob/master/Copy_of_AI_MAJOR_JULY_GOOGLECLASS_SATHEESH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow

In [ ]:
import cv2,os

In [ ]:
data_path="/content/drive/My Drive/dataset"

In [ ]:
label_dict={"with_mask":0,"without_mask":1}

In [ ]:
categories=["with_mask","without_mask"]

In [ ]:
labels=[0,1]

In [ ]:
data=[]
target=[]

In [ ]:
for category in categories:
    folder_path=os.path.join(data_path, category)
    img_names=os.listdir(folder_path)
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(100,100))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            pass

In [ ]:
import numpy as np

In [ ]:
data=np.array(data)

In [ ]:
data=data/255

In [ ]:
data.shape

(1376, 100, 100)

In [ ]:
data=np.reshape(data,(data.shape[0],100,100,1))

In [ ]:
data.shape

(1376, 100, 100, 1)

In [ ]:
target=np.array(target)

In [ ]:
target.shape

(1376,)

In [ ]:
from keras.utils import np_utils

In [ ]:
new_target=np_utils.to_categorical(target)

In [ ]:
new_target.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Dropout
from keras.layers import Conv2D,MaxPooling2D

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(100,(3,3),input_shape=data.shape[1:],activation="relu"))

In [ ]:
model.add(Conv2D(200,(3,3),activation="relu"))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dropout(0.1))

In [ ]:
model.add(Dense(100,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint=ModelCheckpoint("model-{epoch:03d}.model",monitor="val_loss",verbose=0,save_best_only=True,mode="auto")

In [ ]:
history=model.fit(train_data,train_target,epochs=7,validation_split=0.2,callbacks=[checkpoint])

Epoch 1/7
31/31 [==============================] - ETA: 0s - loss: 2.4190 - accuracy: 0.6061WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-001.model/assets
31/31 [==============================] - 269s 9s/step - loss: 2.4190 - accuracy: 0.6061 - val_loss: 0.5004 - val_accuracy: 0.8065
Epoch 2/7
31/31 [==============================] - 268s 9s/step - loss: 0.3526 - accuracy: 0.8465 - val_loss: 0.3234 - val_accuracy: 0.8548
Epoch 3/7
31/31 [==============================] - 257s 8s/step - loss: 0.2099 - accuracy: 0.9222 - val_loss

In [ ]:
print(model.evaluate(test_data,test_target))

NameError: ignored

In [ ]:
 
from keras.models import load_model
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
model=load_model("/content/model-005.model")

OSError: ignored

In [ ]:
face_cascader=cv2.CascadeClassifier("/content/haarcascade_frontalface_default.xml")

In [ ]:
cap=cv2.VideoCapture("/content/satheeshphoto.jpg")
labal_dict={0:"with_mask",1:"without_mask"}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):
    _,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascader.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalised=resized/255.0
        reshaped=np.reshape(normalised,(1,100,100,1))
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img,labal_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    
    cv2_imshow(img)
    key=cv2.waitKey(200)
    if(key==3000):
        break
cv2.distroyAllWindows()
cap.release()